## 업무 자동화 - 웹 크롤링

주피터로 정리하는게 가독성 측면에서 나을 것 같아서 새로 만듬. <br>
가능하면 큰 묶음 별로 cell을 나눠서 **cell 별로 실행 가능**하게 해주고 그에 맞는 설명 달면서 작성해주자. <br>
***기본적으로 조건이 걸리는 부분은 하나의 cell로 해주어야 함.*** (*아래에 쪼갠건 수정할 때 다시 합치셈.*) <br>
예시는 아래에 import 부분만 작성함.

#### 1) 활용 패키지 import 
selenium : 웹 크롤링 <br>
beautifulsoup : html 출력 포맷 변환 <br>
pandas : dataframe 형식 사용 <br>
re : 정규식 사용 <br> 
datetime : 시간 형식 사용 <br>

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  8 21:03:36 2017

@author: HS
"""

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import datetime

<font color='blue'>
수정 코드

아래 코드는 원래 코드에서 내가 필요하다고 생각하는 부분 약간 손봤으니 참고하셈 

(공통 사항), (변경 사항)이라고 괄호 쳐 놓은 부분은 추후에 클래스, 매서드 만들 때 고려될 사항들임. 

코드 비교하면서 보고 변경점이 이해되는 지 알려주세~


In [105]:
# Chrome driver를 실행시킨다. (공통 사항)
driver = wb.Chrome(r'C:\Users\Whi Kwon\Downloads\chromedriver.exe')
driver.implicitly_wait(2)

# 날짜 관련 변수 지정 (공통 사항)
today = datetime.date.today()
week_ago = today - datetime.timedelta(7)
week_day_ago = today - datetime.timedelta(8)

#조달청계약요청 (이후에 업무에 따라 변경될 사항들)
driver.get("http://www.g2b.go.kr:8091/cm/contstus/fwdPpsItemContractReqStus.do")
driver.find_element_by_xpath('//*[@id="prodNm"]').send_keys('전반')
driver.find_element_by_xpath('//*[@id="srchFrm"]/div[2]/div/a[1]/span').click()

## 최종적으로 확인할 리스트들 지정 (공통 사항)
date_sum = []
money_sum = []
number_sum = []
region_sum = []
name_sum = []
needs_name_sum = []

## 반대의 값들을 List로 정리 / Excel로 변환하여 최종 확인할 때 사용 (공통 사항)
date_sum_no = []
money_sum_no = []
number_sum_no = []
region_sum_no = []
name_sum_no = []
needs_name_sum_no = []

## 지역 패턴을 만들어 준다 (공통 사항)
pattern = re.compile(r'대전|충청|충북|충남|세종')

## 앞의 발주(공고)기관명, 뒤의 전화번호만 가져오기 위한 're'함수 사용 
phone_number = re.compile(r'Tel\s*:\s*(\d{2})')
click = 0
#### page 수가 변동 가능한 부분이기에 더 확실하게 해주기 위해서 while문을 사용한다. 
while click is not None:
    
    ## 소스를 가져와 soup으로 정리
    html = driver.page_source
    soup = bs(html, 'lxml')
    #### 'class':'default'가 하나 밖에 없는 값이므로 find로 '더보기'를 찾는다. 넘어갈게 없을 경우 None. 
    click = soup.find(attrs = {'class':'default'})

    # 정보를 가져와서 각각 변수에 저장, 선언.
    date = [j.text.strip() for j in soup.find_all(attrs = {'class':'tc'}) if '/' in j.text]
    # date의 경우 아래에서 위에서 바꿔줘도 되서 여기서 바꿔준다. 뒤에 date() 추가해서 날짜로 맞춰줌. datetime()과 date()는 비교 연산 불가. 
    date = [datetime.datetime.strptime(i, '%Y/%m/%d').date() for i in date]
    money = [j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text]
    money = [int(i.replace('원','').replace(',','')) for i in money]
    number = [j.text.strip() for j in soup.find_all(name = 'a', attrs = {'href' : '#'})][1:]
    region = [j.text.strip() for j in soup.find_all(attrs = {'class':'tl'})][1::2]

    ## 조건을 하나로 합쳐본다 : 날짜가 원하는 날짜가 안나오고, 금액이 1억이 넘으면 클릭
    ## 클릭해서 원하는 지역의 조건이 맞는지 확인해본다(매번 클릭할 때마다 html 새로 지정)
    for k in range(len(date)):        
        
        if date[k] <= week_ago:
            break           
            
        elif money[k] >= 1e8:
            driver.find_element_by_link_text(number[k]).click()
            html = driver.page_source
            soup = bs(html, 'lxml')
            
            ## 발주(공고)기관명, 전화번호만 확인하기 위해 미리 변수 설정
            tem = [j.text.strip() for j in soup.find_all(name = 'td') if 'Tel' in j.text][0]
            split_text = tem.split('(')
            needs_name = split_text[0].strip()
            logic1 = re.search(pattern, soup.find(attrs = {'class':'tl'}).text.strip())
            logic2 = re.search(pattern, soup.find(attrs = {'colspan':'5'}).text.strip())
            logic3 = (re.search(phone_number, split_text[1]) == '04')
            
            
            ## 첫번째로 '납품장소'를 확인한다
            ## 지역이름이 들어갔는지 확인한다            
            ## 그다음 두번째로는 '공사명'을 확인한다
            ## 이 또한 지역이름이 들어갔는지 확인한다
            ## 두 개를 하나의 'if'문으로 통합

            ### 그리고 안 되면 어떻게하면 될지 고민해보자. 
            if  logic1 or logic2:                
                ## '접수번호', '발주(공고)기관의 기관명(전화번호 제외)', '공사명', '단가', '접수일자', '지역'을 추가
                date_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '/' in j.text][0])
                money_sum.append([j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text][0])
                number_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '00' in j.text][0])
                name_sum.append(soup.find(attrs = {'colspan':'5'}).text.strip())
                needs_name_sum.append(needs_name)
            
            ## 마지막으로 '발주(공고)기관'의 전화번호를 확인한다
            ## 전화번호만 확인하는 방법에 대해서 질문
            elif logic3:                
                ## '접수번호', '발주(공고)기관의 기관명(전화번호 제외)', '공사명', '단가', '접수일자', '지역'을 추가
                date_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '/' in j.text][0])
                money_sum.append([j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text][0])
                number_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '00' in j.text][0])
                ## 지역은 위에 질문
                region_sum
                name_sum.append(soup.find(attrs = {'colspan':'5'}).text.strip())
                needs_name_sum.append(needs_name)
            
            ## 1억 미만은 볼 것도 없으니까, 1억 이상의 지역이 아닌 걸로 판명났던거 엑셀로 변환
            else:             
                date_sum_no.append([j.text.strip() for j in soup.find_all(name = 'td') if '/' in j.text][0])
                money_sum_no.append([j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text][0])
                number_sum_no.append([j.text.strip() for j in soup.find_all(name = 'td') if '00' in j.text][0])
                name_sum_no.append(soup.find(attrs = {'colspan':'5'}).text.strip())
                needs_name_sum_no.append(needs_name)
            
            ## '뒤로가기' 버튼을 눌러 그 다음 날짜 및 금액을 확인하러 간다
            driver.back()
            
    if click is not None:
        driver.find_element_by_link_text(click.text).click()

## 아닌 것들을 Excel로 만들기
result_no = pd.DataFrame(data = [number_sum_no,name_sum_no, money_sum_no, needs_name_sum_no, date_sum_no],
                      index = ['접수번호', '공사명', '금액', '공고기관', '접수일자']).T
result_no.to_excel('C:/Users/Whi Kwon/Documents/나라장터_조달청계약요청_아닌것들.xls', index = False)

## 맞는 것들을 Excel로 만들기(확인용)
result = pd.DataFrame(data = [number_sum,name_sum, money_sum, needs_name_sum, date_sum],
                      index = ['접수번호', '공사명', '금액', '공고기관', '접수일자']).T
result.to_excel('C:/Users/Whi Kwon/Documents/나라장터_조달청계약요청.xls', index = False)

## 원래 코드

In [3]:
# Chrome driver를 실행시킨다. 
driver = wb.Chrome(r'C:\Users\Whi Kwon\Downloads\chromedriver.exe')
driver.implicitly_wait(2)

# 날짜 관련 변수 지정
today = datetime.date.today()
week_ago = today - datetime.timedelta(7)
week_day_ago = today - datetime.timedelta(8)
today = today.strftime('%Y/%m/%d')
week_ago = week_ago.strftime('%Y/%m/%d')
week_day_ago = week_day_ago.strftime('%Y/%m/%d')

#조달청계약요청
driver.get("http://www.g2b.go.kr:8091/cm/contstus/fwdPpsItemContractReqStus.do")
driver.find_element_by_xpath('//*[@id="prodNm"]').send_keys('전반')
driver.find_element_by_xpath('//*[@id="srchFrm"]/div[2]/div/a[1]/span').click()

## 최종적으로 확인할 리스트들 지정
date_sum = []
money_sum = []
number_sum = []
region_sum = []
name_sum = []
needs_name_sum = []

## 반대의 값들을 List로 정리 / Excel로 변환하여 최종 확인할 때 사용
date_sum_no = []
money_sum_no = []
number_sum_no = []
region_sum_no = []
name_sum_no = []
needs_name_sum_no = []

## 지역 패턴을 만들어 준다
pattern = re.compile(r'대전|충청|충북|충남|세종')

## 앞의 발주(공고)기관명, 뒤의 전화번호만 가져오기 위한 're'함수 사용
phone_number = re.compile(r'Tel\s*:\s*(\d*)')

## 총 5개의 페이지(최대)로 설정
pages = 5


## 최대 5개의 페이로 지정한 부분을 for문으로 돌림. 전체 페이지를 돌리지 않고, 일정 조건에 부합하면 break 할 예정
for i in range(pages):
    
    ## 소스를 가져와 soup으로 정리
    html = driver.page_source
    soup = bs(html, 'lxml')
    
    ## '클릭' 포함하여 총 6개의 원하는 '일회성' 정보를 가져옴
    ## 추가적으로 date 라는 리스트를 생성하여, 원하는 날짜까지의 정보만 필터
    ## ex) date_tem이 30개라도 date는 원하는 날짜에 멈추므로 17개가 되면, 그 17개에 한해서
    ## 2억 또는 그 외 다른 조건들을 탐색
    date_tem = [j.text.strip() for j in soup.find_all(attrs = {'class':'tc'}) if '/' in j.text]
    money = [j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text]
    money = [int(i.replace('원','').replace(',','')) for i in money]
    number = [j.text.strip() for j in soup.find_all(name = 'a', attrs = {'href' : '#'})][1:]
    region = [j.text.strip() for j in soup.find_all(attrs = {'class':'tl'})][1::2]
    click = [j.text.strip() for j in soup.find_all(attrs = {'class':'default'})]
    ## 'date'라는 List를 만들어 'date_tem'을 날짜 형식으로 아래에서 바꿀 예정
    date = []

    ## 조건을 하나로 합쳐본다 : 날짜가 원하는 날짜가 안나오고, 금액이 1억이 넘으면 클릭
    ## 클릭해서 원하는 지역의 조건이 맞는지 확인해본다(매번 클릭할 때마다 html 새로 지정)
    for k in range(len(date_tem)):
        
        ## 부등호로 적용시키기 위해 date_tem을 날짜 형식으로 바꿈 - 완성
        ## 매 페이지마다 일회성으로 받아오는 데이트를 날짜 형식으로 바꾸면 되므로 여기서 바꾼다
        date.append(datetime.datetime.strptime(date_tem[k], '%Y/%m/%d').strftime('%Y/%m/%d'))
        
        if date[k] <= datetime.datetime.strptime('2017/05/31', '%Y/%m/%d').strftime('%Y/%m/%d'):
            break
            
        elif money[k] >= 1e8:
            driver.find_element_by_link_text(number[k]).click()
            html = driver.page_source
            soup = bs(html, 'lxml')
            
            ## 발주(공고)기관명, 전화번호만 확인하기 위해 미리 변수 설정
            tem = [j.text.strip() for j in soup.find_all(name = 'td') if 'Tel' in j.text]
            split_text = tem[0].split('(')
            needs_name = split_text[0].strip()
            phone =  re.findall(phone_number, tem[0])
            
            ## 첫번째로 '납품장소'를 확인한다
            ## 지역이름이 들어갔는지 확인한다            
            ## 그다음 두번째로는 '공사명'을 확인한다
            ## 이 또한 지역이름이 들어갔는지 확인한다
            ## 두 개를 하나의 'if'문으로 통합

            ### if 문 or로 합치는 건 좋은 생각. re.search 내부에서 or를 걸면 안 될거임. 다른 코드로 확인해보자. 
            ### 그리고 안 되면 어떻게하면 될지 고민해보자. 
            if re.search(pattern, soup.find(attrs = {'class':'tl'}).text.strip()) or\
            re.search(pattern, soup.find(attrs = {'colspan':'5'}).text.strip()):
                
                ## '접수번호', '발주(공고)기관의 기관명(전화번호 제외)', '공사명', '단가', '접수일자', '지역'을 추가
                date_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '/' in j.text][0])
                money_sum.append([j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text][0])
                number_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '00' in j.text][0])
                ## 지역은 위에 질문
                region_sum
                name_sum.append(soup.find(attrs = {'colspan':'5'}).text.strip())
                needs_name_sum.append(needs_name)
            
            ## 마지막으로 '발주(공고)기관'의 전화번호를 확인한다
            ## 전화번호만 확인하는 방법에 대해서 질문
            elif '04' in phone[0]:
                
                ## '접수번호', '발주(공고)기관의 기관명(전화번호 제외)', '공사명', '단가', '접수일자', '지역'을 추가
                date_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '/' in j.text][0])
                money_sum.append([j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text][0])
                number_sum.append([j.text.strip() for j in soup.find_all(name = 'td') if '00' in j.text][0])
                ## 지역은 위에 질문
                region_sum
                name_sum.append(soup.find(attrs = {'colspan':'5'}).text.strip())
                needs_name_sum.append(needs_name)
            
            ## 생각해보니 'Naver'에 검색할 필요가 없다 (나라장터에서 할 3가지 업무 중 이번 업무는) 이유는 하기와 같다
            ## 실질적으로 Naver에 검색할 것은 발주(공고)기관인데, 이는 전화번호로 정확히 확인했기 때문!
            ## 따라서 Naver 검색을 하지 않고, 아니었던 List는 Excel로 만들어서 확인해본다
            ## 1억 미만은 볼 것도 없으니까, 1억 이상의 지역이 아닌 걸로 판명났던거 엑셀로 변환
            else:
                
                date_sum_no.append([j.text.strip() for j in soup.find_all(name = 'td') if '/' in j.text][0])
                money_sum_no.append([j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text][0])
                number_sum_no.append([j.text.strip() for j in soup.find_all(name = 'td') if '00' in j.text][0])
                ## 지역은 위에 질문
                region_sum_no
                name_sum_no.append(soup.find(attrs = {'colspan':'5'}).text.strip())
                needs_name_sum_no.append(needs_name)
            
            ## '뒤로가기' 버튼을 눌러 그 다음 날짜 및 금액을 확인하러 간다
            driver.back()

    
    ## '클릭(+더보기)' 버튼을 누름
    ## 클릭 버튼이 없을 경우 break를 할 수 있는 조건문 생성
    if click == []:
        break
    else:
        driver.find_element_by_link_text(click[0]).click()

## 아닌 것들을 Excel로 만들기
result_no = pd.DataFrame(data = [number_sum_no,name_sum_no, region_sum_no, money_sum_no, needs_name_sum_no, date_sum_no],
                      index = ['접수번호', '공사명', '지역', '금액', '공고기관', '접수일자']).T
result_no.to_excel('C:/Users/Whi Kwon/Documents/나라장터_조달청계약요청_아닌것들.xls', index = False)

## 맞는 것들을 Excel로 만들기(확인용)
result = pd.DataFrame(data = [number_sum,name_sum, region_sum, money_sum, needs_name_sum, date_sum],
                      index = ['접수번호', '공사명', '지역', '금액', '공고기관', '접수일자']).T
result.to_excel('C:/Users/Whi Kwon/Documents/나라장터_조달청계약요청.xls', index = False)

<font color = 'red'>
2가지 문제점을 발견!<br><br>
1) 휘 너가 지적해서 안됐던 것들은 알고보니 Cell을 합치니 되었다.<br>
     ex) for 문이 이어지고 있는 상황에서 for 문 내의 if 문을 다른 cell에서 실행하게 되면,<br>
         정확한 원인은 모르겠지만, 안됨<br>
         이거에 대한 문제점 해결은 단순히 cell을 합치니 실행이 되는 것으로 확인<br><br>
   또한, re.search 문제점은 휘 너말대로 or가 작동하지 않음 후후후 구굿<br>
   따라서, re.search를 그냥 2번 쓰는 걸로 그냥 바꿈!(이거는 잘 작동 확인 완료)<br>


2) 두번째 문제점은 'click'에 대한 것이다<br>
   현재 계속 수정하면서 진행하고 있지만, 'click'이 [] 이렇게만 나올 경우, list index out of range로 뜬다<br>
   따라서 이 문제점은 click을 수정하면서 진행 중임! -> 수정완료<br><br>
   
3) 휘 완성했어!<br>
   날짜는 확인용으로 하기 위해 일부러 길게 잡았어~<br>
   또한, 엑셀 파일을 2개 만들었어 (조건에 맞는 것, 조건에 아닌 것들 / 단, 1억 이상만)<br>
   확인 결과 모두 맞게 나왔어 검토 부탁할게!<br>

굳굳 고생했음 <br>
먼저 코드에 대한 피드백 주고 아래에 잘 돌아가는 지 확인하는 방법에 대해서 설명할게 <br>

1) **Feedback**
    - region_sum : 위에서 보니 안 쓰이는 변수처럼 보임. (*결과에서도 지역이 빠지고 출력됨*), 필요 없을 시 삭제.     
    
2) **추가 필요 사항** <br>
    - 입력 변수 따로 빼서 관리 (*입력해주거나 변할 수도 있는 것들.*) 
    - 너무 긴 실행 작업 함수로 만들어서 관리 (*가독성, 코드 관리를 위함.*) 
    - 2~3가지 유사한 작업 진행 시 공통 부분을 묶어서 클래스화 관리 필요. (*중복되는 작업을 막고 관리의 용이성을 위함*) 
→ 이 사항들은 내가 가이드라인 제시해줄테니 일단은 코드 완성도에 집중. 시간되면 내가 올려놓은 Python_class라는 파일 보면 됨.

3) **코드 검증** <br>
여기는 순차적으로 확인하다고 보면 됨. 주로 코드 중간에 print를 써서 확인하는 방법, assert를 써서 확인하는 방법이 있고 
마지막 출력 결과에 대해서는 엑셀로 확인하면 될 듯함. 

1. print : 말 그대로 코드 중간에 지나갈 때 데이터 개수나 값을 출력해서 본다. 값을 다 볼 수 없으니 보통 데이터의 형식이나 개수 확인. <br> 
2. assert : print가 돌아갈 때의 상황을 확인하다고 하면 assert는 에러가 뜨는지 보는 함수이다. 마찬가지로 데이터의 형식이나 개수를 주로 확인하며 계속 변동이 있는 **입력값**에 많이 써서 오입력을 방지한다. 아래 예시처럼 사용 가능.
```python
assert 1 > 2, "부등호가 잘못되었습니다."
```
    **확인 필요 사항들**
    - 코드 진행 중 저장되는 데이터(1p에 30개) 개수 확인. 아마 당연히 맞을거지만 확인하는 방법 학습을 위해서 코드 중간 중간에 print넣어서 
    ***아 코드가 이렇게 동작하고 있구나, 데이터 확인이 되는구나 *** 를 알기 위해 한 번 해보자. 모든 데이터에 대해서 수행하는게 좋고 
    실제 나중에 사용할 때에도 돌아가는 진행사항 확인을 위해서 넣어주는 게 좋음. (*맨 아래 항목과 매칭*)
    - 코드 동작 후 저장되어 있는 데이터 개수 (True/False) 확인. 
    - 원하는 로직대로 수행이 되었는 지 확인
    - 날짜를 좀 더 길게해서 작업 진행 (1년?)
    (*1년이나 하는 이유는 1년간 변동이 없이 잘 되었다면 앞으로 큰 변동이 없을 것이라는 생각.*)
    - 웹 페이지에 변동이 만약 생겼을 경우에 어떻게 확인할 지에 대한 알림 로직 추가 필요
    (*아마 페이지가 변경되어도 데이터가 끌어와지면 원하는 결과는 얻을 수 있을테니 데이터 수나 형식에 대한 알람 로직을 걸면 될듯.*)
    
</font>

In [100]:
'''
    우선 if문으로 지정한 날짜가 맞으면 멈춤!
    if date[j] == week_day_ago:
        break
    
    금액이 2억이 넘으면 클릭한다!
    저번 숙제에 쓴대로 우선 2억이 넘으면 모두 클릭할 예정
    if money[j] == 2e8:
        driver.find_element_by_link_text(number[j]).click()
        
        3가지 순서대로 지역에 맞는지 확인
        납품장소 - 공사명 - 발주(공고)기관
        또한, 각 공고 내의 위 사항들은 모두 같은 xpath를 가짐을 확인
        
        if 납품장소 == "대전~~~":
            필요한 항목들을 끌고옴
            
        else if 공사명 == "대전~~~~":
            필요한 항목들을 끌고옴
            
        else if 발주(공고)기관 == "041", "042", "043", "044":
            필요한 항목들을 끌고옴
            
        else if 발주(공고)기관을 '네이버지도'에 검색하여 상위 3개를 확인하여 "대전~~~"이 있으면:
            필요한 항목들을 끌고옴
            
        else:
            2억 이상의 리스트를 확인할 수 있도록 엑셀로 만들어서 확인
            이건 그때 그때 마다 보고 지울 생각.
            또한, 원하는 지역이 아니었던 발주(공고)기관들은 리스트로 만들어서
            추후에 네이버 지도에 검색하는 것보다 우선적으로 리스트를 확인할 수 있도록 만들고 싶음
    
    내가 원하는 결론은
    1) 필요한 항목들을 엑셀로 만들어서 보는 것
    2) 내가 걸러낸 항목(2억 이하)에 대해서 맞는지 확인
       -> 그래서 납품장소, 공사명도 가져와야 되는지 의문..
    3) 내가 걸러낸 발주(공고)기관들은 계속 데이터로 가지고 있을 예정 '''


                    

IndentationError: unexpected indent (<ipython-input-100-6d156ed6e2e8>, line 3)